In [1]:
import transformers as t
import datasets
from datasets import load_dataset

import torch
import torch_geometric
from torch_geometric.data import Data

import numpy as np
import pandas as pd

import spacy

from sklearn.preprocessing import LabelEncoder

c:\Users\Frank\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
q_train = load_dataset("maastrichtlawtech/bsard", data_files="questions_fr_train.csv", split="train")
q_test = load_dataset("maastrichtlawtech/bsard", data_files="questions_fr_test.csv", split="train")
articles = load_dataset("maastrichtlawtech/bsard", data_files="articles_fr.csv", split="train")

Found cached dataset csv (C:/Users/Frank/.cache/huggingface/datasets/maastrichtlawtech___csv/maastrichtlawtech--bsard-d4b4993551724e5a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Found cached dataset csv (C:/Users/Frank/.cache/huggingface/datasets/maastrichtlawtech___csv/maastrichtlawtech--bsard-41a15be090989dc9/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Found cached dataset csv (C:/Users/Frank/.cache/huggingface/datasets/maastrichtlawtech___csv/maastrichtlawtech--bsard-1ac8feb008d877a3/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


In [26]:
articles[1000]["article"]

"Dans les quinze jours du prononcé, le greffier notifie le dispositif du jugement aux parties par pli judiciaire. Le délai d'appel est de deux mois à dater de cette notification. L'appel est formé par requête à la chambre de la famille de lacour d'appel. II doit, à peine de nullité, être dénoncé par exploit d'huissier ou par lettre recommandée dans les huit jours à dater de la réception de la requête, au greffe du tribunal de première instancequi a rendu la décision attaquée. Le greffier fait mention du recours en marge de la décision attaquée. Les règles prévues pour la première instance s'appliquent au degré d'appel.L'arrêt est notifié par le greffier de la cour d'appel conformément à ce qui est prévu pour la première instance. Le délai pour se pourvoir en cassation est d'un mois à dater de cette notification.Le délai pour se pourvoir en cassation et le pourvoi contre l'arrêt constatant le décès sont suspensifs."

In [15]:
q_train

DatasetDict({
    train: Dataset({
        features: ['id', 'category', 'subcategory', 'question', 'extra_description', 'article_ids'],
        num_rows: 886
    })
})

In [20]:
dataset = load_dataset("FranklinWillemen/mapa_plus")
df_train = pd.DataFrame(dataset["train"])
df_val = pd.DataFrame(dataset["validation"])
df_test = pd.DataFrame(dataset["test"])

Found cached dataset parquet (C:/Users/Frank/.cache/huggingface/datasets/FranklinWillemen___parquet/FranklinWillemen--mapa_plus-4bbc20eeb61798b9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 3/3 [00:00<00:00, 602.80it/s]


In [24]:
" ".join(df_train['tokens'][3000])

'A la luz de las consideraciones anteriores , propongo al Tribunal de Justicia que responda a las cuestiones prejudiciales planteadas por la Corte suprema di cassazione ( Tribunal Supremo de Casación , Italia ) del siguiente modo : « El artículo 35 , apartado 3 , del Reglamento ( CE ) n.o 44 / 2001 del Consejo , de 22 de diciembre de 2000 , relativo a la competencia judicial , el reconocimiento y la ejecución de resoluciones judiciales en materia civil y mercantil , y el artículo 24 del Reglamento ( CE ) n.o 2201 / 2003 del Consejo , de 27 de noviembre de 2003 , relativo a la competencia , el reconocimiento y la ejecución de resoluciones judiciales en materia matrimonial y de responsabilidad parental , por el que se deroga el Reglamento ( CE ) n.o 1347 / 2000 , deben interpretarse en el sentido de que prohíben que la infracción de las normas de litispendencia , previstas en el artículo 27 del Reglamento n.o 44 / 2001 y en el artículo 19 del Reglamento n.o 2201 / 2003 , por parte del ór